In [1]:
import pandas as pd 
import sklearn as sk
import numpy as np

In [2]:
train_set=pd.read_csv("train.csv",encoding='utf-8')

In [3]:
test_set=pd.read_csv("test.csv",encoding='utf-8')

In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM,GRU
from tensorflow.keras.utils import to_categorical

#se hace un análisis del target usando NLP
X_train=train_set['text'].copy().values
Y_train=train_set['target'].copy().values
X_test=test_set['text'].copy().values

#pruebo usando un word embedding que yo entreno
#práctica NO recomendada por la catedra, pero
#quiero ver que resultados puedo conseguir con un
#embedding mio. Hasta ahora el puntaje mio no 
#alcanza a 0.7
print("Shape de X_train: "+ str(X_train.shape))
print("Shape de X_test:" + str(X_test.shape))
todos_los_textos=np.concatenate([X_train,X_test])
print("Shape de todos_los_textos:" + str(todos_los_textos.shape))

#estan todos los textos concatenados, hay que 'entrenar' al tokenizador
objeto_tokenizador=Tokenizer()
objeto_tokenizador.fit_on_texts(todos_los_textos)

#necesito una cota de la longitud de cada palabra de los textos que 
#se van a analizar
longitud_maxima=max([len(s.split()) for s in todos_los_textos])
print("longitud maxima: " + str(longitud_maxima))

#necesito saber cuantas palabras tengo en mi 'diccionario' de palabras
tamanio_de_vocabulario=len(objeto_tokenizador.word_index) +1
print("tamanio de vocabulario: " + str(tamanio_de_vocabulario))

#ahora que tengo esto, es tiempo de tokenizar cada uno de los tweets
#y agregar el padding necesario
X_train_tokens=objeto_tokenizador.texts_to_sequences(X_train)
X_train_pad=pad_sequences(X_train_tokens,maxlen=longitud_maxima,padding='post')

X_test_tokens=objeto_tokenizador.texts_to_sequences(X_test)
X_test_pad=pad_sequences(X_test_tokens,maxlen=longitud_maxima,padding='post')

#ahora hay que generar el modelo, se va a usar una red neuronal de 3 capas 
modelo=Sequential()
modelo.add(Embedding(tamanio_de_vocabulario,100,input_length=longitud_maxima))
modelo.add(GRU(units=32, dropout=0.2 ,recurrent_dropout=0.2))
modelo.add(Dense(1,activation='sigmoid'))

modelo.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
modelo.summary()

/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

Shape de X_train: (7613,)
Shape de X_test:(3263,)
Shape de todos_los_textos:(10876,)
longitud maxima: 31
tamanio de vocabulario: 29320
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 31, 100)           2932000   
_________________________________________________________________
gru (GRU)                    (None, 32)                12768     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,944,801
Trainable params: 2,944,801
N

In [5]:
y_binary=to_categorical(Y_train)

In [6]:
#se entrena el modelo:
print("Train...")
modelo.fit(X_train_pad,Y_train,batch_size=128,epochs=25)

Train...
Epoch 1/25
7613/7613 [==============================] - 7s 906us/sample - loss: 0.6839 - acc: 0.5702
Epoch 2/25
7613/7613 [==============================] - 5s 678us/sample - loss: 0.6835 - acc: 0.5706
Epoch 3/25
7613/7613 [==============================] - 5s 694us/sample - loss: 0.6084 - acc: 0.6643
Epoch 4/25
7613/7613 [==============================] - 5s 643us/sample - loss: 0.4003 - acc: 0.8518
Epoch 5/25
7613/7613 [==============================] - 5s 649us/sample - loss: 0.2541 - acc: 0.9158
Epoch 6/25
7613/7613 [==============================] - 5s 661us/sample - loss: 0.1696 - acc: 0.9468
Epoch 7/25
7613/7613 [==============================] - 5s 647us/sample - loss: 0.1043 - acc: 0.9712
Epoch 8/25
7613/7613 [==============================] - 5s 649us/sample - loss: 0.0596 - acc: 0.9849
Epoch 9/25
7613/7613 [==============================] - 5s 640us/sample - loss: 0.0419 - acc: 0.9904
Epoch 10/25
7613/7613 [==============================] - 5s 657us/sample - loss: 0

In [7]:
#ahora se hace la prediccion
predicciones=modelo.predict(X_test_pad)

In [8]:
predicciones.shape

(3263, 1)

In [9]:
#dado que no es un modelo recomendado por la catedra, voy a asumir que las predicciones
#son muy poco confiables, entonces solo voy a asignarle si son verdaderos los tweets
#si el modelo cree muy fuertemente en su predicción, caso contrario, lo considero falso
lista=[]
for elemento in predicciones:
    if(elemento>=0.95):
        lista.append(1)
    else:
        lista.append(0)

In [10]:
len(lista)

3263

In [12]:
sampleSubmission=pd.read_csv('sample_submission.csv')

In [13]:
output=pd.DataFrame({'id':sampleSubmission.id,'target':lista})

In [14]:
output.to_csv('submission.csv',index=False)